In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from flask import Flask, jsonify
from sqlalchemy import create_engine, func
from datetime import datetime
import pandas as pd

In [2]:
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(autoload_with=engine)
# reflect the tables
Base.classes.keys()

# Save references to each table
station = Base.classes.station
measurement = Base.classes.measurement


In [3]:
app = Flask(__name__)
session = Session(engine)

In [4]:
start_date = "2010-1-7"
end_date = "2010-4-10"

In [10]:
start_object = datetime.strptime(start_date, "%Y-%m-%d").date()
end_object = datetime.strptime(end_date, "%Y-%m-%d").date()

In [11]:
print(start_object)
print(end_object)

2010-01-07
2010-04-10


In [6]:
tobs_from = session.query(
    measurement.date, 
    func.min(measurement.tobs).label('min_1'),
    func.avg(measurement.tobs).label('avg_1'),
    func.max(measurement.tobs).label('max_1')
).filter(
    measurement.date >= start_date,
    measurement.date <= end_date
).group_by(
    measurement.date
).all() 

In [12]:
tobs_from = session.query(measurement.date, 
                              func.min(measurement.tobs),\
                              func.avg(measurement.tobs),\
                                func.max(measurement.tobs)).\
                                filter(measurement.date >= start_date).\
                                filter(measurement.date <= end_date).\
                                group_by(measurement.date)

print(tobs_from)

SELECT measurement.date AS measurement_date, min(measurement.tobs) AS min_1, avg(measurement.tobs) AS avg_1, max(measurement.tobs) AS max_1 
FROM measurement 
WHERE measurement.date >= ? AND measurement.date <= ? GROUP BY measurement.date


In [14]:
tobs_from_to_list = []
for d, min, avg, max in tobs_from:
        tobs_from_data = {}
        tobs_from_data["Date"] = d
        tobs_from_data["Minimum Temperature"] = min
        tobs_from_data["Average Temperature"] = round(avg, 1)
        tobs_from_data["Maximum Temperature"] = max
        tobs_from_to_list.append(tobs_from_data)
print(tobs_from_to_list)

[{'Date': '2010-10-01', 'Minimum Temperature': 71.0, 'Average Temperature': 73.0, 'Maximum Temperature': 78.0}, {'Date': '2010-10-02', 'Minimum Temperature': 73.0, 'Average Temperature': 76.8, 'Maximum Temperature': 80.0}, {'Date': '2010-10-03', 'Minimum Temperature': 70.0, 'Average Temperature': 76.6, 'Maximum Temperature': 82.0}, {'Date': '2010-10-04', 'Minimum Temperature': 71.0, 'Average Temperature': 74.8, 'Maximum Temperature': 78.0}, {'Date': '2010-10-05', 'Minimum Temperature': 71.0, 'Average Temperature': 76.4, 'Maximum Temperature': 80.0}, {'Date': '2010-10-06', 'Minimum Temperature': 71.0, 'Average Temperature': 75.8, 'Maximum Temperature': 79.0}, {'Date': '2010-10-07', 'Minimum Temperature': 68.0, 'Average Temperature': 74.7, 'Maximum Temperature': 80.0}, {'Date': '2010-10-08', 'Minimum Temperature': 69.0, 'Average Temperature': 75.7, 'Maximum Temperature': 81.0}, {'Date': '2010-10-09', 'Minimum Temperature': 72.0, 'Average Temperature': 76.4, 'Maximum Temperature': 79.0}, 

In [26]:
blah = session.query(measurement.date, measurement.tobs, measurement.prcp)

In [27]:
tester = pd.DataFrame(blah)

In [28]:
any_nulls = tester['prcp'].isnull().values.any()
print(any_nulls)

True


In [ ]:
@app.route("/api/v1.0/start_date/end_date/<start_date>/<end_date>")
def start_end(start_date, end_date):
    
    session = Session(engine)

    tobs_from = session.query(measurement.date, 
                              func.min(measurement.tobs),\
                              func.avg(measurement.tobs),\
                                func.max(measurement.tobs)).\
                                filter(measurement.date >= start_date).\
                                filter(measurement.date <= end_date).\
                                group_by(measurement.date)


    session.close()

    print(tobs_from)

    tobs_from_to_list = []
    for d, min, avg, max in tobs_from:
        tobs_from_data = {}
        tobs_from_data["Date"] = d
        tobs_from_data["Minimum Temperature"] = min
        tobs_from_data["Average Temperature"] = round(avg, 1)
        tobs_from_data["Maximum Temperature"] = max
        tobs_from_to_list.append(tobs_from_data)

    

    return jsonify(tobs_from_to_list)

